In [27]:
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

In [28]:
view_line_range = (0, 10)


import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143
The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! yo

In [29]:
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    text_set = list(set(text))
    vocab_to_int = {word:i for (i, word) in enumerate(text_set)}
    int_to_vocab = {i:word for (word, i) in vocab_to_int.items()}

    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [30]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
        
    return {
         '.':  '<period>',
         ',':  '<comma>',
         '"':  '<quotation_mark>',
         ';':  '<semicolon>',
         '!':  '<exclamation_mark>',
         '?':  '<question_mark>',
         '(':  '<left_parentheses>',
         ')':  '<right_parentheses>',
         '-':  '<dash>',
         '\n': '<return>',
    }


Tests Passed


In [31]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [32]:
int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [33]:
import torch

train_on_gpu = torch.cuda.is_available()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [34]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    X = []
    y = []
    for i in range(sequence_length, len(words)-1):
        X.append(words[i-sequence_length:i])
        y.append(words[i])
    X = np.array(X)
    y = np.array(y)

    data = TensorDataset(torch.from_numpy(X), torch.from_numpy(y))
    loader = DataLoader(data, shuffle=True, batch_size=batch_size)

    return loader



In [35]:
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[38, 39, 40, 41, 42],
        [ 1,  2,  3,  4,  5],
        [ 4,  5,  6,  7,  8],
        [27, 28, 29, 30, 31],
        [20, 21, 22, 23, 24],
        [ 6,  7,  8,  9, 10],
        [34, 35, 36, 37, 38],
        [17, 18, 19, 20, 21],
        [26, 27, 28, 29, 30],
        [40, 41, 42, 43, 44]])

torch.Size([10])
tensor([43,  6,  9, 32, 25, 11, 39, 22, 31, 45])


In [36]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """

        super(RNN, self).__init__()
        # set class variables
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                                dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out[:,-1,:]
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


In [37]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    # move data to GPU, if available
    inp = inp.to(device)
    target = target.to(device)
    
    optimizer.zero_grad()
    output, hidden = rnn.forward(inp, hidden)
    hidden = tuple([h.data for h in hidden])

    loss = criterion(output, target)

    # perform backpropagation and optimization
    
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden


Tests Passed


In [38]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    return rnn

In [39]:
# Data params
# Sequence Length
sequence_length = 25  # of words in a sequence
# Batch Size
batch_size = 500

# data loader
train_loader = batch_data(int_text, sequence_length, batch_size)

In [40]:
# Training parameters
# Number of Epochs
num_epochs = 8
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 1000
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

In [41]:
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 8 epoch(s)...
Epoch:    1/8     Loss: 5.45239008808136

Epoch:    1/8     Loss: 4.596855948448181

Epoch:    1/8     Loss: 4.369500174999237

Epoch:    2/8     Loss: 4.187580264952718

Epoch:    2/8     Loss: 4.0788312029838565

Epoch:    2/8     Loss: 4.0395884289741515

Epoch:    3/8     Loss: 3.916087165778997

Epoch:    3/8     Loss: 3.8512970666885376

Epoch:    3/8     Loss: 3.8394798707962035

Epoch:    4/8     Loss: 3.736192768325611

Epoch:    4/8     Loss: 3.683756896018982

Epoch:    4/8     Loss: 3.6770925736427307

Epoch:    5/8     Loss: 3.5878706659589494

Epoch:    5/8     Loss: 3.549227572917938

Epoch:    5/8     Loss: 3.5432490034103394

Epoch:    6/8     Loss: 3.4550640053895054

Epoch:    6/8     Loss: 3.401256357192993

Epoch:    6/8     Loss: 3.423488987445831

Epoch:    7/8     Loss: 3.318705863794502

Epoch:    7/8     Loss: 3.300695508480072

Epoch:    7/8     Loss: 3.3179827361106873

Epoch:    8/8     Loss: 3.2064647644150015

Epoch:    8/8     

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

In [43]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [44]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

elaine:" the game of the new york"

jerry:(still laughing) i mean...(he exits, and starts looking at the door.)

george:(on phone) i don't know, but i got a little bit for you.

george: oh, no. no! no, no, no, no, no, no, no, no, no. no! no. no, no. i'm not...

jerry: no, no. i don't know how i can. i can't do this. i'm gonna go.

elaine: what are you doing?

george: i'm gonna go see the movie.

jerry: i don't even know what i was thinking.

jerry:(still laughing) i don't think so.

elaine: oh, i can't.

george:(looking at jerry) hey, what are you doing here?(he looks at the door, then sees the door) i was in the bathroom.

jerry: i know, i'm sorry, but i don't think so.

jerry: oh, you know what? i don't know. i mean, if you were going to the movies, and we could do this for the rest of my life. i think i can get some of those things.(elaine enters.) what?!

jerry: hey, what are you doing?

kramer: i'm sorry. i got a date with a man.

elaine: yeah!

kramer: well, i'm sure that i'm not

In [45]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()